In [1]:
import pandas as pd 
import numpy as np


df_rating = pd.read_csv("C:\\Users\\DarkZone\\Desktop\\Data science\\Movie Recommendation System\\Netflix_Dataset_Rating.csv")
df_movies = pd.read_csv("C:\\Users\\DarkZone\\Desktop\\Data science\\Movie Recommendation System\\Netflix_Dataset_Movie.csv")

   


In [2]:
df_movies.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [3]:
df_rating.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


In [4]:
print('movies df dimension:', df_movies.shape, 'rating df dimension:', df_rating.shape)


movies df dimension: (17770, 3) rating df dimension: (17337458, 3)


In [5]:
movies_name = df_movies.set_index('Movie_ID')['Name'].to_dict() # create a dictionary of movie id and movie name
n_users = df_rating['User_ID'].nunique() # number of unique users
n_movies = df_rating['Movie_ID'].nunique() # number of unique movies
print('Number of users:', n_users, 'Number of movies:', n_movies)
print('Full rating matrix shape:', n_users * n_movies)
print('number of ratings:', len(df_rating)) # number of ratings
print('sparsity:', len(df_rating) / (n_users * n_movies) * 100, '%') # sparsity of the matrix

Number of users: 143458 Number of movies: 1350
Full rating matrix shape: 193668300
number of ratings: 17337458
sparsity: 8.952140334788915 %


In [6]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_movies, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.movie_factors = torch.nn.Embedding(n_movies, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.movie_factors.weight.data.uniform_(0, 0.05)


    def forward(self, user, movie):
        return (self.user_factors(user) * self.movie_factors(movie)).sum(1)



    def predict(self, user, movie):
        return self.forward(user, movie)

        










In [17]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness


class Loader(Dataset):
    def __init__ (self):
        self.ratings = df_rating.copy()  # copy the dataframe

        users = df_rating.User_ID.unique() # get unique users
        movies = df_rating.Movie_ID.unique() # get unique movies

        self.user2idx = {user: i for i, user in enumerate(users)}  # create a dictionary of user id and index
        self.movie2idx = {movie: i for i, movie in enumerate(movies)} # create a dictionary of movie id and index


         # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.user2idx.items()}  # create a dictionary of index and user id
        self.idx2movieid = {i:o for o,i in self.movie2idx.items()}  # create a dictionary of index and movie id


        self.ratings.Movie_ID = self.ratings.Movie_ID.apply(lambda x: self.movie2idx[x]) # map movie id to index
        self.ratings.User_ID = self.ratings.User_ID.apply(lambda x: self.user2idx[x]) # map user id to index    

        self.x = self.ratings.drop('Rating' , axis=1).values # get the features
        self.y = self.ratings['Rating'].values # get the target
        self.x , self.y = torch.tensor(self.x, dtype=torch.long), torch.tensor(self.y, dtype=torch.float32) # convert to tensor


    def __getitem__(self, index) :        
        return (self.x[index], self.y[index])  # return the features and the target
    

    def __len__(self):
        return len(self.ratings)   # return the length of the dataset
    
    
      


    



In [18]:
num_epochs = 128
cuda = torch.cuda.is_available() # check if cuda is available
print(cuda)


model = MatrixFactorization(n_users, n_movies, n_factors=20) # create the model

for name , param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)   # print the parameters of the model

    if cuda:
        model = model.cuda() # move the model to the GPU


    lose_fn = torch.nn.MSELoss() # define the loss function



    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # define the optimizer


    train_set = Loader() # create the dataloader

    train_loader = DataLoader(train_set,128 , shuffle=True) # create the dataloader  
    
          

False
user_factors.weight tensor([[0.0231, 0.0386, 0.0124,  ..., 0.0166, 0.0367, 0.0077],
        [0.0274, 0.0056, 0.0250,  ..., 0.0151, 0.0298, 0.0128],
        [0.0236, 0.0402, 0.0399,  ..., 0.0426, 0.0476, 0.0036],
        ...,
        [0.0368, 0.0199, 0.0126,  ..., 0.0051, 0.0135, 0.0034],
        [0.0451, 0.0048, 0.0442,  ..., 0.0489, 0.0018, 0.0155],
        [0.0361, 0.0150, 0.0306,  ..., 0.0369, 0.0399, 0.0156]])
movie_factors.weight tensor([[6.8039e-05, 4.7333e-03, 2.7172e-02,  ..., 1.8532e-02, 2.2026e-03,
         2.0176e-02],
        [2.0117e-02, 3.8949e-02, 1.7534e-02,  ..., 4.4263e-03, 1.0268e-02,
         9.6092e-03],
        [1.1716e-02, 2.9428e-02, 1.8670e-02,  ..., 4.5511e-02, 1.1407e-02,
         2.3127e-02],
        ...,
        [4.5068e-02, 5.8657e-03, 4.9602e-04,  ..., 1.5289e-02, 1.6128e-02,
         2.5384e-02],
        [3.0773e-02, 2.3904e-02, 2.7412e-02,  ..., 7.0789e-03, 2.0184e-02,
         3.7906e-02],
        [9.2982e-03, 3.3421e-02, 4.3743e-02,  ..., 4.0112

In [19]:
for it in tqdm(range(num_epochs)):
    losses = []         # create a list to store the loss

for x , y in train_loader:   
    if cuda:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        output = model(x)
        loss = lose_fn(output.squeeze(), y.type(torch.float64))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        print('epoch:', it, 'loss:', np.mean(losses))
           

C:\Users\DarkZone\AppData\Local\Temp\ipykernel_14456\579644498.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

In [20]:
# By training the model, we will have tuned latent factors for movies and users.
# We can now use these factors to make predictions.
# Let's see how well the model performs on a few examples.

c= 0
a = 0
b = 0
for name , param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)   # print the parameters of the model

        if c == 0:
           a =param.data  
           c += 1    # print the parameters of the model

        else:
              b = param.data  # print the parameters of the model   





user_factors.weight tensor([[0.0231, 0.0386, 0.0124,  ..., 0.0166, 0.0367, 0.0077],
        [0.0274, 0.0056, 0.0250,  ..., 0.0151, 0.0298, 0.0128],
        [0.0236, 0.0402, 0.0399,  ..., 0.0426, 0.0476, 0.0036],
        ...,
        [0.0368, 0.0199, 0.0126,  ..., 0.0051, 0.0135, 0.0034],
        [0.0451, 0.0048, 0.0442,  ..., 0.0489, 0.0018, 0.0155],
        [0.0361, 0.0150, 0.0306,  ..., 0.0369, 0.0399, 0.0156]])
movie_factors.weight tensor([[6.8039e-05, 4.7333e-03, 2.7172e-02,  ..., 1.8532e-02, 2.2026e-03,
         2.0176e-02],
        [2.0117e-02, 3.8949e-02, 1.7534e-02,  ..., 4.4263e-03, 1.0268e-02,
         9.6092e-03],
        [1.1716e-02, 2.9428e-02, 1.8670e-02,  ..., 4.5511e-02, 1.1407e-02,
         2.3127e-02],
        ...,
        [4.5068e-02, 5.8657e-03, 4.9602e-04,  ..., 1.5289e-02, 1.6128e-02,
         2.5384e-02],
        [3.0773e-02, 2.3904e-02, 2.7412e-02,  ..., 7.0789e-03, 2.0184e-02,
         3.7906e-02],
        [9.2982e-03, 3.3421e-02, 4.3743e-02,  ..., 4.0112e-02, 

In [21]:
trained_movie_factors = model.movie_factors.weight.data.cpu().numpy()   # get the trained movie factors

In [22]:
len(trained_movie_factors) # check the length of the trained movie factors


1350

In [32]:
from sklearn.cluster import KMeans  # import the KMeans algorithm

kmeans = KMeans(n_clusters=10  , random_state=0).fit(trained_movie_factors)  # create the KMeans object and fit it to the trained movie factors  # check the labels of the first 10 movies

In [33]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''

for cluster in range(10):
    print('Cluster#{}'.format(cluster))   # print the cluster number

    movie = []      # create a list to store the movie names
    for movidx in np.where(kmeans.labels_ == cluster)[0]:       # get the movie index
        movid = train_set.idx2movieid[movidx]   # get the movie id
        rat_count = df_rating.loc[df_rating['Movie_ID'] == movid].count()[0]  # get the number of ratings
        movie.append((movies_name[movid], rat_count))  # append the movie name and the number of ratings to the list
    for movie in sorted(movie, key=lambda x: x[1], reverse=True):  # sort the movie by the number of ratings
        print("\t",movie[0])   # print the movie name    



Cluster#0
	 Lord of the Rings: The Fellowship of the Ring
	 Shrek 2
	 The Bourne Supremacy
	 Road to Perdition
	 Sideways
	 Mean Girls
	 Sixteen Candles
	 The Godfather
	 The Lion King: Special Edition
	 Alien: Collector's Edition
	 Frida
	 Four Weddings and a Funeral
	 War Games
	 Superman: The Movie
	 Boyz N the Hood
	 From Hell
	 The Rocky Horror Picture Show
	 25th Hour
	 Roger & Me
	 Bottle Rocket
	 The Ice Storm
	 Commando
	 The Toy
	 The Parent Trap
	 The Way We Were
	 Doctor Zhivago
	 CSI: Season 1
	 The Quiet Man
	 Congo
	 Against the Ropes
	 The Santa Clause
	 Happiness
	 The Lawnmower Man
	 The Limey
	 Fly Away Home
	 Ghost in the Shell
	 Psycho
	 Pollyanna
	 Coffee and Cigarettes
	 Mansfield Park
	 The Caine Mutiny
	 Mean Creek
	 Superstar
	 Beverly Hills Ninja
	 The Brothers McMullen
	 Under Siege 2: Dark Territory
	 Monty Python: The Life of Python
	 Nick of Time
	 Nosferatu: Original Version
	 Lonesome Dove
	 All the Right Moves
	 Spitfire Grill
	 Rose Red
	 The Looney T